In [0]:
import pandas as pd
#pip install openpyxl

In [0]:
Data_Train = pd.read_excel("Data_Train.xlsx")

In [4]:
Data_Train.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [5]:
print(len(Data_Train))
Data_Train.describe()

11094


,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094,11094,11094,11094,11094
unique,7480,35,2179,26,18,33,1103,761,7
top,ID_7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,₹200,₹50,-,-,-,30 minutes
freq,22,947,850,3241,10118,1191,2074,2312,7406


In [0]:
X = Data_Train.copy()

In [0]:
X.dropna(axis=0,subset=['Delivery_Time'],inplace=True)
y= X.Delivery_Time
X.drop(['Delivery_Time'],axis=1,inplace=True)

In [8]:
len(X)

11094

In [9]:
from sklearn.model_selection import train_test_split

#features = ["Rating",	"Votes", "Reviews"]
features = ["Rating","Delivery_Time"]
X_new = Data_Train.copy()
X_rating = X_new[features]
X_rating.dropna(axis=0,subset=features,inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
for row in X_rating:
  print(X_rating[row])

0        3.5
1        3.5
2        3.6
3        3.7
4        3.2
        ... 
11089    4.2
11090    3.6
11091    3.5
11092    3.1
11093    NEW
Name: Rating, Length: 11094, dtype: object
0        30 minutes
1        30 minutes
2        65 minutes
3        30 minutes
4        65 minutes
            ...    
11089    30 minutes
11090    30 minutes
11091    30 minutes
11092    30 minutes
11093    30 minutes
Name: Delivery_Time, Length: 11094, dtype: object


In [0]:
X_rating = X_rating.replace(["NEW","-","Opening Soon",'Temporarily Closed'],2.5)

In [12]:
X_rating

,Rating,Delivery_Time
0,3.5,30 minutes
1,3.5,30 minutes
2,3.6,65 minutes
3,3.7,30 minutes
4,3.2,65 minutes
...,...,...
11089,4.2,30 minutes
11090,3.6,30 minutes
11091,3.5,30 minutes
11092,3.1,30 minutes


In [13]:
y_rating = X_rating.Delivery_Time
X_rating.drop(["Delivery_Time"],axis=1,inplace=True)
X_rating.astype("float32").dtypes

Rating    float32
dtype: object

In [14]:
X_rating.shape

(11094, 1)

In [15]:
y_rating

0        30 minutes
1        30 minutes
2        65 minutes
3        30 minutes
4        65 minutes
            ...    
11089    30 minutes
11090    30 minutes
11091    30 minutes
11092    30 minutes
11093    30 minutes
Name: Delivery_Time, Length: 11094, dtype: object

In [16]:
print(y_rating[0].replace("minutes",""))

30 


In [0]:
for col in y_rating.index:
  y_rating[col]=y_rating[col].replace("minutes","")

In [18]:
y_rating

0        30 
1        30 
2        65 
3        30 
4        65 
        ... 
11089    30 
11090    30 
11091    30 
11092    30 
11093    30 
Name: Delivery_Time, Length: 11094, dtype: object

In [19]:
y_rating.astype("int32").dtypes

dtype('int32')

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
model_1 = RandomForestRegressor(n_estimators=500,random_state=1)

In [0]:
X_rating_train,X_rating_valid,y_rating_train,y_rating_valid = train_test_split(X_rating,y_rating)

In [23]:
len(X_rating_train)

8320

In [24]:
model_1.fit(X_rating_train,y_rating_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [0]:
pred = model_1.predict(X_rating_valid)

In [0]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(pred,y_rating_valid)

In [27]:
score

0.001687094448449891

In [0]:
def getScores(n_est,X_train,y_train,X_valid,y_valid):
  scores = {}
  for k in n_est:
    model = RandomForestRegressor(n_estimators=k,random_state=1)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    score = mean_absolute_error(preds,y_valid)
    scores[k]=score
  return scores

In [0]:
n_est = [50,100,150,200,500,1000,1500,2000]
scores = getScores(n_est, X_rating_train, y_rating_train, X_rating_valid, y_rating_valid)

In [30]:
scores

{50: 0.0012977649603460712,
 100: 0.0013698630136986304,
 150: 0.001586157173756308,
 200: 0.0014780100937274692,
 500: 0.001687094448449891,
 1000: 0.001737563085796683,
 1500: 0.0017880317231434735,
 2000: 0.0019214131218457122}

In [0]:
X_rating = X_rating.drop(["Delivery_Time"],axis=1)

In [32]:
X_rating.head()

,Rating
0,3.5
1,3.5
2,3.6
3,3.7
4,3.2


In [35]:
model_test = RandomForestRegressor(n_estimators=100,random_state=1)
model_test.fit(X_rating,y_rating)
X_test = pd.read_excel("Data_Test.xlsx")
X_test = X_test["Rating"]
X_test = X_test.replace(["NEW","-","Opening Soon",'Temporarily Closed'],2.5)
X_test.astype("float32").dtypes

dtype('float32')

In [36]:
X_test

0       4.2
1       2.5
2       3.6
3       3.6
4       2.9
       ... 
2769    2.5
2770    3.1
2771    3.0
2772    3.0
2773    3.6
Name: Rating, Length: 2774, dtype: object

In [0]:
import numpy as np

In [0]:
X_test = np.array(X_test)

In [0]:
X_test = X_test.reshape(-1,1)

In [40]:
X_test.shape

(2774, 1)

In [41]:
X_rating_train.shape

(8320, 2)

In [0]:
predictions_test = model_test.predict(X_test)

In [0]:
import math

In [0]:
pred_min = []
for p in predictions_test:
    time = math.ceil(p)
    d = time // 5
    time = d * 5
    pred_min.append(str(time) + " minutes")

In [0]:
#pred_min

In [0]:
output = pd.DataFrame({'Delivery_Time':pred_min})

In [0]:
#output.to_csv("Submission_test_rating_min.csv",index=False)

Note: you may need to restart the kernel to use updated packages.


In [0]:
output.to_excel("Submission_2.xlsx",index=False)